In [1]:
import ChemSpaceAL
from ChemSpaceAL import InitializeWorkspace
from ChemSpaceAL import Configuration
from ChemSpaceAL import Dataset
from ChemSpaceAL import Model
from ChemSpaceAL import Training
from ChemSpaceAL import Generation
from ChemSpaceAL import Sampling
from ChemSpaceAL import ALConstruction


import os

In [2]:
import numpy as np

np.__version__

'1.26.2'

In [3]:
# Intiialize the workspace directory
base_path = os.getcwd() + "/publication_runs/"
InitializeWorkspace.create_folders(base_path=base_path)
config = Configuration.Config(
    base_path=base_path,
    cycle_prefix="model0",
    al_iteration=1,
    cycle_suffix="ch1",
    training_fname="moses_train.csv.gz",
    validation_fname="moses_test.csv.gz",
    slice_data=1_000,
    verbose=True,
)
config.set_previous_arrays()
# ds = Dataset.load_data(config=config, mode="Pretraining")

    will create folders at base_path='/Users/morgunov/batista/Summer/ChemSpaceAL/publication_runs/'
--- The following previously scored molecules were set:
     4_Scoring/scored_dataframes/model0_al0_ch1.csv
--- The following previously constructed Active Learning sets were set:
     5_ActiveLearning/training_sets/model0_al0_ch1.csv


In [4]:
config.verbose = True
config.set_training_parameters(mode="Pretraining", epochs=1)
config.set_generation_parameters(
    target_criterion="force_number_filtered",
    force_filters="ADMET+FGs",
    target_number=1_000,
)

config.verbose = True
config.set_sampling_parameters(
    n_clusters=10,
    samples_per_cluster=2,
    pca_fname="scaler_pca_combined_processed_freq1000_block133_120.pkl",
)
config.set_scoring_parameters(protein_path="1ieb_processed.pdb")
config.set_active_learning_parameters(
    selection_mode="threshold", threshold=11, training_size=10_000
)
# model, trainer = Training.train_GPT(
#     config=config,
#     training_dataset=ds[0],
# )
# Generation.generate_smiles(config)
# Generation.characterize_generated_molecules(config)

--- The following training parameters were set:
    number of epochs: 1
    learning rate: 0.0003
    learning warmup enabled? True
    model weights will be saved to:               1_Pretraining/model_weights/model0_al1_ch1.pt                                   
    dataset descriptors will be loaded from:      1_Pretraining/datasets_descriptors/moses_train.yaml                             
  . note: wandb_project_name and wandb_runname were not provided, you can ignore this message if you don't plan to log runs to wandb
--- The following generation parameters were set:
    target number: 1000 unique canonical smiles that pass filters
    batch size: 64 & temperature: 1.0
    the following filters will be applied: ADMET+FGs
    model weights will be loaded from:            1_Pretraining/model_weights/model0_al1_ch1.pt                                   
    dataset descriptors will be loaded from:      1_Pretraining/datasets_descriptors/moses_train.yaml                             
    

In [5]:
# Sampling.calculate_descriptors(config)
# mols = Sampling.project_into_pca_space(config)
# Sampling.cluster_and_sample(mols=mols, config=config, n_iter=1)
# ALConstruction.construct_al_training_set(config=config, do_sampling=True)

In [6]:
config.set_active_learning_parameters(
    selection_mode="threshold",
    probability_mode="linear",
    threshold=11,
    training_size=10,
)

--- The following AL training set construction parameters were set:
    the training set will be constructed to have 10 molecules
    of which 5 will be selected from the top scoring molecules defined by the following parameters:
        molecules with score above 11 will be selected
    the remaining 5 molecules will be selected from high-scoring clusters according to the following parameters:
        the following probability mode will be used: linear
    the training set will be saved to             5_ActiveLearning/training_sets/model0_al1_ch1.csv                               


In [7]:
al_ds = Dataset.load_data(config=config, mode="Active Learning")

Will load AL training set from 5_ActiveLearning/training_sets/model0_al0_ch1.csv


In [8]:
config.set_training_parameters(mode="Active Learning", epochs=1)

--- The following training parameters were set:
    number of epochs: 1
    learning rate: 3e-05
    learning warmup enabled? False
    model weights will be loaded from:            1_Pretraining/model_weights/model0_al0_ch1.pt                                   
    model weights will be saved to:               5_ActiveLearning/model_weights/model0_al1_ch1.pt                                
    dataset descriptors will be loaded from:      1_Pretraining/datasets_descriptors/moses_train.yaml                             
  . note: wandb_project_name and wandb_runname were not provided, you can ignore this message if you don't plan to log runs to wandb


In [9]:
model, trainer = Training.train_GPT(
    config=config,
    training_dataset=al_ds,
)

/Users/morgunov/.pyenv/versions/3.11.4/lib/python3.11/site-packages/torch/cuda/amp/grad_scaler.py:125: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(
  0%|          | 0/1 [00:05<?, ?it/s]


KeyboardInterrupt: 

In [18]:
config.cycle_temp_params["path_to_al_training_set"]

'/Users/morgunov/batista/Summer/ChemSpaceAL/publication_runs/5_ActiveLearning/training_sets/model0_al0_ch1.csv'

In [21]:
"/Users/morgunov/batista/Summer/ChemSpaceAL/publication_runs/5_ActiveLearning/training_sets/model0_al0_ch1.csv".replace(
    f"al0", f"al1"
)

'/Users/morgunov/batista/Summer/ChemSpaceAL/publication_runs/5_ActiveLearning/training_sets/model0_al1_ch1.csv'